In [1]:
import logging
import os

import numpy as np

from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.stats import (
    check_model_preference_aic,
    copy_target_config,
    fetch_all_analysis_fit_info,
    get_model_config_files,
    tabulate_best_fit_stats,
)

In [2]:
# os.environ['GAMMAPY_DATA']

In [3]:
log = logging.getLogger("Check Preferred Spectral Model")

In [4]:
config_file = "../src/asgardpy/tests/config_test_base.yaml"

# Initiate the main "base" config

In [5]:
main_config = AsgardpyConfig.read(config_file)
target_source_name = main_config.target.source_name

In [6]:
spec_model_temp_files = get_model_config_files(["lp", "bpl", "ecpl", "pl", "eclp"])

In [7]:
main_config.dataset3d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}fermipy-crab/"
main_config.dataset3d.instruments[0].input_dl3[1].input_dir = f"{os.environ['GAMMAPY_DATA']}fermipy-crab/"

main_config.dataset1d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}hess-dl3-dr1/"
main_config.dataset1d.instruments[1].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}magic/rad_max/data/"

In [8]:
steps_list = []
for s in main_config.general.steps:
    if s != "flux-points":
        steps_list.append(s)

In [9]:
steps_list

['datasets-3d', 'datasets-1d', 'fit']

In [10]:
def fetch_all_analysis_objects(main_config, spec_model_temp_files):
    """For a list of spectral models, initiate AsgardpyAnalysis objects."""
    main_analysis_list = {}
    spec_models_list = []

    for temp in spec_model_temp_files:
        temp_model = AsgardpyAnalysis(main_config)
        temp_model.config.target.models_file = temp

        temp_model_2 = AsgardpyAnalysis(temp_model.config)

        copy_target_config(temp_model, temp_model_2)

        spec_tag = temp.name.split(".")[0].split("_")[-1]
        spec_models_list.append(spec_tag)
        main_analysis_list[spec_tag] = {}

        main_analysis_list[spec_tag]["Analysis"] = temp_model_2

    spec_models_list = np.array(spec_models_list)

    return main_analysis_list, spec_models_list

In [11]:
%%time
main_analysis_list, spec_models_list = fetch_all_analysis_objects(main_config, spec_model_temp_files)

Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filem

CPU times: user 58.4 ms, sys: 5.39 ms, total: 63.8 ms
Wall time: 63.4 ms


In [12]:
spec_models_list

array(['lp', 'bpl', 'ecpl', 'pl', 'eclp'], dtype='<U4')

# Run Analysis steps

In [13]:
%%time
PL_idx = 0

for i, tag in enumerate(spec_models_list):
    log.info("Spectral model being tested: %s", tag)

    main_analysis_list[tag]["Analysis"].run(steps_list)

    if tag == "pl":
        PL_idx = i

Spectral model being tested: lp
Perform DL3 to DL4 process!
1 number of 3D Datasets given
The different keys used: 00 01
Selecting the file with name containing 00
Selecting the file with name containing 00
Selecting the file with name containing 00
Selecting the file with name containing 00
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set MJD-OBS to 58484.000000 from DATE-OBS.
Set MJD-END to 59729.988334 from DATE-END'. [astropy.wcs.wcs]
Set MJD-OBS to 58484.000000 from DATE-OBS.
Set MJD-END to 59729.988334 from DATE-END'.
Using the exclusion mask to create a safe mask
Selecting the file with name containing 01
Selecting the file with name containing 01
Selecting the file with name containing 01
Selecting the file with name containing 01
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa.
Set MJD-OBS to 58484.000000 from DATE-OBS.
Set MJD-END to 59

CPU times: user 4min 13s, sys: 2.68 s, total: 4min 16s
Wall time: 4min 18s


# Get all the Fit results

In [14]:
%%time
fit_success_list, stat_list, dof_list, pref_over_pl_chi2_list = fetch_all_analysis_fit_info(
    main_analysis_list, spec_models_list
)

CPU times: user 685 μs, sys: 979 μs, total: 1.66 ms
Wall time: 1.82 ms


In [15]:
# If any spectral model has at least 5 sigmas preference over PL
best_sp_idx_lrt = np.nonzero(pref_over_pl_chi2_list == np.nanmax(pref_over_pl_chi2_list))[0]
for idx in best_sp_idx_lrt:
    if pref_over_pl_chi2_list[idx] > 5:
        sp_idx_lrt = idx
        log.info("Best preferred spectral model over PL is %s", spec_models_list[idx])
    else:
        sp_idx_lrt = PL_idx
        log.info("No other model preferred over PL")

Best preferred spectral model over PL is ecpl


In [16]:
list_rel_p = check_model_preference_aic(stat_list, dof_list)

In [17]:
best_sp_idx_aic = np.nonzero(list_rel_p == np.nanmax(list_rel_p))[0]

for idx in best_sp_idx_aic:
    if list_rel_p[idx] > 0.95:
        sp_idx_aic = idx
        log.info("Best preferred spectral model is %s", spec_models_list[fit_success_list][idx])
    else:
        sp_idx_aic = PL_idx
        log.info("No other model preferred, hence PL is selected")

No other model preferred, hence PL is selected


# Write the results into a Table

In [18]:
stats_table = tabulate_best_fit_stats(spec_models_list, fit_success_list, main_analysis_list, list_rel_p)

stats_table.meta["Target source name"] = target_source_name

In [19]:
stats_table

Spectral Model,TS of Best Fit,TS of Max Fit,TS of Goodness of Fit,DoF of Fit,Significance ($\sigma$) of Goodness of Fit,p-value of Goodness of Fit,Pref over PL (chi2),Pref over PL (p-value),Pref over PL (DoF),Relative p-value (AIC)
str4,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64
LP,-4441.766,-4520.08,78.315,72,1.068,0.2854,6.979,2.98e-12,1,0.09075
BPL,-4392.942,-4520.08,127.139,72,3.988,6.651e-05,0.0,1.0,1,2.269e-12
ECPL,-4442.291,-4520.08,77.789,72,1.037,0.2997,7.016,2.279e-12,1,0.118
PL,-4393.064,-4520.08,127.017,73,3.909,9.267e-05,0.0,0.0,0,8.873e-13
ECLP,-4444.096,-4520.08,75.984,71,0.992,0.3211,6.833,8.287e-12,2,0.7912
